In [4]:
import pandas as pd
import numpy as np
from pyspi.calculator import Calculator
import os.path as op
from copy import deepcopy
import argparse

subject_id='100307'
data_path='/headnode1/abry4213/data/HCP100'
github_dir='/headnode1/abry4213/github/Directed_information_fMRI'
SPI_subset=f'{github_dir}/feature_extraction/pyspi/fast_extended.yaml'
connectivity_type = 'homotopic' 
ROI_lookup_file = f'{github_dir}/Brain_Region_info.csv'

In [5]:

# Get the base name for SPI_subset file
SPI_subset_base = op.basename(SPI_subset).replace(".yaml", "")

# Time series output path for this subject
time_series_path = f"{data_path}/raw_data/functional_MRI/numpy_files/"
output_feature_path = op.join(data_path, "time_series_features/pyspi")

# Define ROI lookup table
brain_region_lookup = pd.read_csv(ROI_lookup_file)

# Check if pyspi results already exist for this subject
output_file = f"{output_feature_path}/HCP100_{subject_id}_all_pyspi_{SPI_subset_base}_results.csv"
if op.isfile(output_file):
    print(f"{SPI_subset_base} SPI results for {subject_id} already exist. Skipping.")
    exit() 

# Load in the .npy file
npy_file = f"{time_series_path}/{subject_id}.npy"
TS_data = np.load(npy_file)


In [ ]:
if connectivity_type=='homotopic':
    homotopic_region_res_list = []
    for brain_region_base in brain_region_lookup.Base_Region[0:1]:
        left_index = brain_region_lookup[brain_region_lookup.Brain_Region == f"ctx-lh-{brain_region_base}"].Region_Index.values[0]
        right_index = brain_region_lookup[brain_region_lookup.Brain_Region == f"ctx-rh-{brain_region_base}"].Region_Index.values[0]

        left_right_TS_only = TS_data[[left_index, right_index], :]

        # Make deepcopy of calc 
        calc_copy = deepcopy(calc)

        # Load data 
        calc_copy.load_dataset(left_right_TS_only)

        # Compute SPIs
        calc_copy.compute()

        # Get SPI results
        SPI_res = deepcopy(calc_copy.table)

        SPI_res.columns = SPI_res.columns.to_flat_index()
        SPI_res = SPI_res.rename(columns='__'.join).assign(hemi_from = lambda x: x.index)

        # Reshape from wide to long, and add columns for hemisphere and base region
        SPI_res_long = SPI_res.melt(id_vars='hemi_from', var_name='SPI__hemi_to', value_name='value')
        SPI_res_long["SPI"] = SPI_res_long["SPI__hemi_to"].str.split("__").str[0]
        SPI_res_long = (SPI_res_long
                        .assign(hemi_from = lambda x: np.where(x.hemi_from == 'proc-0', 'left', 'right'),
                                hemi_to = lambda x: np.where(x.SPI__hemi_to.str.contains('proc-0'), 'left', 'right'),
                                base_region_to = brain_region_base,
                                base_region_from = brain_region_base)
                        .drop(columns='SPI__hemi_to')
                        .query("hemi_from != hemi_to")
                        )
        
        # Append to list
        homotopic_region_res_list.append(SPI_res_long)

    # Concatenate all homotopic region results
    homotopic_region_res = pd.concat(homotopic_region_res_list)

In [33]:
connectivity_type='homotopic' 
calc = Calculator(configfile=SPI_subset)

homotopic_region_res_list = []
for brain_region_base in brain_region_lookup.Base_Region[0:1]:
    left_index = brain_region_lookup[brain_region_lookup.Brain_Region == f"ctx-lh-{brain_region_base}"].Region_Index.values[0]
    right_index = brain_region_lookup[brain_region_lookup.Brain_Region == f"ctx-rh-{brain_region_base}"].Region_Index.values[0]

    left_right_TS_only = TS_data[[left_index, right_index], :]

    # Make deepcopy of calc 
    calc_copy = deepcopy(calc)

    # Load data 
    calc_copy.load_dataset(left_right_TS_only)

    # Compute SPIs
    calc_copy.compute()

    # Get SPI results
    SPI_res = deepcopy(calc_copy.table)

    SPI_res.columns = SPI_res.columns.to_flat_index()
    SPI_res = SPI_res.rename(columns='__'.join).assign(hemi_from = lambda x: x.index)

    # Reshape from wide to long, and add columns for hemisphere and base region
    SPI_res_long = SPI_res.melt(id_vars='hemi_from', var_name='SPI__hemi_to', value_name='value')
    SPI_res_long["SPI"] = SPI_res_long["SPI__hemi_to"].str.split("__").str[0]
    SPI_res_long = (SPI_res_long
                    .assign(hemi_from = lambda x: np.where(x.hemi_from == 'proc-0', 'left', 'right'),
                            hemi_to = lambda x: np.where(x.SPI__hemi_to.str.contains('proc-0'), 'left', 'right'),
                            base_region_to = brain_region_base,
                            base_region_from = brain_region_base)
                    .drop(columns='SPI__hemi_to')
                    .query("hemi_from != hemi_to")
                    )
    
    # Append to list
    homotopic_region_res_list.append(SPI_res_long)

# Concatenate all homotopic region results
homotopic_region_res = pd.concat(homotopic_region_res_list)

Frequency minimum set to 0; overriding to 1e-5.


Loading configuration file: /headnode1/abry4213/github/Directed_information_fMRI/feature_extraction/pyspi/fast_extended.yaml
*** Importing module .statistics.basic
[0] Adding SPI .statistics.basic.Covariance(x,y,{'estimator': 'EmpiricalCovariance'})
Succesfully initialised SPI with identifier "cov_EmpiricalCovariance" and labels ['basic', 'unordered', 'linear', 'undirected', 'signed']
[1] Adding SPI .statistics.basic.Covariance(x,y,{'estimator': 'GraphicalLasso'})
Succesfully initialised SPI with identifier "cov_GraphicalLasso" and labels ['basic', 'unordered', 'linear', 'undirected', 'signed']
[2] Adding SPI .statistics.basic.Covariance(x,y,{'estimator': 'GraphicalLassoCV'})
Succesfully initialised SPI with identifier "cov_GraphicalLassoCV" and labels ['basic', 'unordered', 'linear', 'undirected', 'signed']
[3] Adding SPI .statistics.basic.Covariance(x,y,{'estimator': 'LedoitWolf'})
Succesfully initialised SPI with identifier "cov_LedoitWolf" and labels ['basic', 'unordered', 'linear'

Processing [None: bary_sgddtw_mean]:  21%|██        | 54/256 [00:04<00:20, 10.08it/s]           DBA loss is increasing while it should not be. Stopping optimization.
Processing [None: phase_multitaper_mean_fs-1_fmin-0_fmax-0-5]:  45%|████▍     | 114/256 [00:20<00:47,  3.02it/s]Mean of empty slice
Processing [None: phase_multitaper_max_fs-1_fmin-0_fmax-0-5]:  45%|████▍     | 114/256 [00:20<00:47,  3.02it/s]    All-NaN slice encountered
Processing [None: sgc_nonparametric_mean_fs-1_fmin-0_fmax-0-5]:  80%|███████▉  | 204/256 [00:23<00:02, 21.61it/s]   Mean of empty slice
Processing [None: sgc_nonparametric_max_fs-1_fmin-0_fmax-0-5]:  80%|███████▉  | 204/256 [00:23<00:02, 21.61it/s]    All-NaN slice encountered
Processing [None: pec_orth_log_abs]: 100%|██████████| 256/256 [00:23<00:00, 10.76it/s]                                      



Calculation complete. Time taken: 23.7943s

SPI Computation Results Summary

Total number of SPIs attempted: 256
Number of SPIs successfully computed: 254 (99.22%)
------------------------------------------------------------
Category       | Count | Percentage
------------------------------------------------------------
Successful     |   254 |  99.22%
NaNs           |     2 |   0.78%
Partial NaNs   |     0 |   0.00%
------------------------------------------------------------

[2] SPI(s) produced NaN outputs:
------------------------------------------------------------
1. gd_multitaper_delay_fs-1_fmin-0_fmax-0-5
2. gd_multitaper_delay_fs-1_fmin-0-25_fmax-0-5
------------------------------------------------------------



In [17]:
brain_region_lookup

,Region_Index,Brain_Region,ggseg,Hemisphere,Base_Region
0,0,ctx-lh-bankssts,ctx-lh-bankssts,Left,bankssts
1,1,ctx-lh-caudalanteriorcingulate,ctx-lh-caudalanteriorcingulate,Left,caudalanteriorcingulate
2,2,ctx-lh-caudalmiddlefrontal,ctx-lh-caudalmiddlefrontal,Left,caudalmiddlefrontal
3,3,ctx-lh-cuneus,ctx-lh-cuneus,Left,cuneus
4,4,ctx-lh-entorhinal,ctx-lh-entorhinal,Left,entorhinal
...,...,...,...,...,...
63,63,ctx-rh-supramarginal,ctx-rh-supramarginal,Right,supramarginal
64,64,ctx-rh-frontalpole,ctx-rh-frontalpole,Right,frontalpole
65,65,ctx-rh-temporalpole,ctx-rh-temporalpole,Right,temporalpole
66,66,ctx-rh-transversetemporal,ctx-rh-transversetemporal,Right,transversetemporal


In [ ]:
SPI_res_long = SPI_res.melt(id_vars='hemi_from', var_name='SPI__hemi_to', value_name='value')


In [ ]:

SPI_res_long["hemi_to"] = SPI_res_long["SPI__hemi_to"].str.split("__").str[1]

SPI_res_long = (SPI_res_long.rename(columns={"hemi_from": "he_from_index", "region_to": "region_to_index"})
                    .reset_index(drop=True)
                    .assign(region_from_index = lambda x: x.region_from_index.str.replace("proc-", ""),
                        region_to_index = lambda x: x.region_to_index.str.replace("proc-", ""))
                    .astype({"region_from_index": int, "region_to_index": int})
                    .merge(brain_region_lookup[["Region_Index", "Base_Region", "Hemisphere"]], left_on="region_from_index", right_on="Region_Index", how="left")
                    .rename(columns={"Base_Region": "base_region_from", "Hemisphere": "hemi_from"})
                    .drop(columns=["Region_Index"])
                    .merge(brain_region_lookup[["Region_Index", "Base_Region", "Hemisphere"]], left_on="region_to_index", right_on="Region_Index", how="left")
                    .rename(columns={"Base_Region": "base_region_to", "Hemisphere": "hemi_to"})
                    .drop(columns=["Region_Index", "region_from_index", "region_to_index", "SPI__region_to"])
                    # .query("base_region_from == base_region_to & hemi_from != hemi_to")
                )
